In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import random

In [2]:
data = pd.read_csv("emo_day365.csv")
data = data[['emotion_id']].values
dataset = data.astype('float32')

In [9]:
data

array([[1],
       [1],
       [4],
       [1],
       [3],
       [3],
       [3],
       [3],
       [2],
       [3],
       [4],
       [5],
       [1],
       [5],
       [3],
       [5],
       [4],
       [1],
       [4],
       [3],
       [2],
       [1],
       [2],
       [4],
       [3],
       [3],
       [2],
       [1],
       [5],
       [5],
       [3],
       [2],
       [5],
       [1],
       [4],
       [3],
       [3],
       [2],
       [2],
       [5],
       [5],
       [5],
       [1],
       [5],
       [1],
       [1],
       [4],
       [2],
       [3],
       [2],
       [1],
       [2],
       [4],
       [4],
       [3],
       [5],
       [1],
       [1],
       [4],
       [2],
       [5],
       [1],
       [4],
       [4],
       [1],
       [2],
       [5],
       [1],
       [4],
       [5],
       [4],
       [3],
       [5],
       [3],
       [3],
       [5],
       [3],
       [1],
       [5],
       [4],
       [2],
       [2],
       [3],
    

In [3]:
dataset

array([[1.],
       [1.],
       [4.],
       [1.],
       [3.],
       [3.],
       [3.],
       [3.],
       [2.],
       [3.],
       [4.],
       [5.],
       [1.],
       [5.],
       [3.],
       [5.],
       [4.],
       [1.],
       [4.],
       [3.],
       [2.],
       [1.],
       [2.],
       [4.],
       [3.],
       [3.],
       [2.],
       [1.],
       [5.],
       [5.],
       [3.],
       [2.],
       [5.],
       [1.],
       [4.],
       [3.],
       [3.],
       [2.],
       [2.],
       [5.],
       [5.],
       [5.],
       [1.],
       [5.],
       [1.],
       [1.],
       [4.],
       [2.],
       [3.],
       [2.],
       [1.],
       [2.],
       [4.],
       [4.],
       [3.],
       [5.],
       [1.],
       [1.],
       [4.],
       [2.],
       [5.],
       [1.],
       [4.],
       [4.],
       [1.],
       [2.],
       [5.],
       [1.],
       [4.],
       [5.],
       [4.],
       [3.],
       [5.],
       [3.],
       [3.],
       [5.],
       [3.],

In [4]:
# scaler = MinMaxScaler(feature_range=(1, 50))
# dataset = scaler.fit_transform(dataset)

In [5]:
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]

In [6]:
from numpy import array
 
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


In [21]:
n_steps = 7
# split into samples
X, y = split_sequence(dataset, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[[1.]
 [1.]
 [4.]
 [1.]
 [3.]
 [3.]
 [3.]] [3.]
[[1.]
 [4.]
 [1.]
 [3.]
 [3.]
 [3.]
 [3.]] [2.]
[[4.]
 [1.]
 [3.]
 [3.]
 [3.]
 [3.]
 [2.]] [3.]
[[1.]
 [3.]
 [3.]
 [3.]
 [3.]
 [2.]
 [3.]] [4.]
[[3.]
 [3.]
 [3.]
 [3.]
 [2.]
 [3.]
 [4.]] [5.]
[[3.]
 [3.]
 [3.]
 [2.]
 [3.]
 [4.]
 [5.]] [1.]
[[3.]
 [3.]
 [2.]
 [3.]
 [4.]
 [5.]
 [1.]] [5.]
[[3.]
 [2.]
 [3.]
 [4.]
 [5.]
 [1.]
 [5.]] [3.]
[[2.]
 [3.]
 [4.]
 [5.]
 [1.]
 [5.]
 [3.]] [5.]
[[3.]
 [4.]
 [5.]
 [1.]
 [5.]
 [3.]
 [5.]] [4.]
[[4.]
 [5.]
 [1.]
 [5.]
 [3.]
 [5.]
 [4.]] [1.]
[[5.]
 [1.]
 [5.]
 [3.]
 [5.]
 [4.]
 [1.]] [4.]
[[1.]
 [5.]
 [3.]
 [5.]
 [4.]
 [1.]
 [4.]] [3.]
[[5.]
 [3.]
 [5.]
 [4.]
 [1.]
 [4.]
 [3.]] [2.]
[[3.]
 [5.]
 [4.]
 [1.]
 [4.]
 [3.]
 [2.]] [1.]
[[5.]
 [4.]
 [1.]
 [4.]
 [3.]
 [2.]
 [1.]] [2.]
[[4.]
 [1.]
 [4.]
 [3.]
 [2.]
 [1.]
 [2.]] [4.]
[[1.]
 [4.]
 [3.]
 [2.]
 [1.]
 [2.]
 [4.]] [3.]
[[4.]
 [3.]
 [2.]
 [1.]
 [2.]
 [4.]
 [3.]] [3.]
[[3.]
 [2.]
 [1.]
 [2.]
 [4.]
 [3.]
 [3.]] [2.]
[[2.]
 [1.]
 [2.]
 [4.]
 [3.]
 [3.]
 [2.

 [1.]] [4.]
[[4.]
 [4.]
 [5.]
 [5.]
 [4.]
 [1.]
 [4.]] [4.]
[[4.]
 [5.]
 [5.]
 [4.]
 [1.]
 [4.]
 [4.]] [1.]
[[5.]
 [5.]
 [4.]
 [1.]
 [4.]
 [4.]
 [1.]] [2.]
[[5.]
 [4.]
 [1.]
 [4.]
 [4.]
 [1.]
 [2.]] [3.]
[[4.]
 [1.]
 [4.]
 [4.]
 [1.]
 [2.]
 [3.]] [3.]
[[1.]
 [4.]
 [4.]
 [1.]
 [2.]
 [3.]
 [3.]] [1.]
[[4.]
 [4.]
 [1.]
 [2.]
 [3.]
 [3.]
 [1.]] [2.]
[[4.]
 [1.]
 [2.]
 [3.]
 [3.]
 [1.]
 [2.]] [4.]
[[1.]
 [2.]
 [3.]
 [3.]
 [1.]
 [2.]
 [4.]] [5.]
[[2.]
 [3.]
 [3.]
 [1.]
 [2.]
 [4.]
 [5.]] [3.]
[[3.]
 [3.]
 [1.]
 [2.]
 [4.]
 [5.]
 [3.]] [4.]
[[3.]
 [1.]
 [2.]
 [4.]
 [5.]
 [3.]
 [4.]] [1.]
[[1.]
 [2.]
 [4.]
 [5.]
 [3.]
 [4.]
 [1.]] [5.]
[[2.]
 [4.]
 [5.]
 [3.]
 [4.]
 [1.]
 [5.]] [3.]
[[4.]
 [5.]
 [3.]
 [4.]
 [1.]
 [5.]
 [3.]] [1.]
[[5.]
 [3.]
 [4.]
 [1.]
 [5.]
 [3.]
 [1.]] [4.]
[[3.]
 [4.]
 [1.]
 [5.]
 [3.]
 [1.]
 [4.]] [3.]
[[4.]
 [1.]
 [5.]
 [3.]
 [1.]
 [4.]
 [3.]] [3.]
[[1.]
 [5.]
 [3.]
 [1.]
 [4.]
 [3.]
 [3.]] [3.]
[[5.]
 [3.]
 [1.]
 [4.]
 [3.]
 [3.]
 [3.]] [5.]
[[3.]
 [1.]
 [4.]
 [3.]
 [3.

In [11]:

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [12]:

# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [13]:
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [ ]:
x_input = array(random.randint())
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)

In [20]:
random.randint(1,10)

3